In [1]:
!pip install torch torchvision
!pip install pillow==4.3.0

In [0]:
%matplotlib inline
import torch
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [0]:
vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
  param.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
 81%|████████▏ | 467763200/574673361 [00:04<00:01, 99900582.75it/s]

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg.to(device)

In [0]:
def load_image(img_path, max_size=400, shape=None):
  image = Image.open(img_path).convert('RGB')
  if max(image.size) > max_size:
    size = max_size
  else:
    size = max(image.size)
   
  if shape is not None:
    size = shape
    
  in_transform = transforms.Compose([
      transforms.Resize(size),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
  
  image = in_transform(image).unsqueeze(0)
  
  return image

In [0]:
content = load_image('City.jpg').to(device)
style = load_image('StarryNight.jpg', shape=content.shape[-2:]).to(device)

In [0]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.squeeze()
  image = image.transpose(1, 2, 0)
#   unnormalise
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(im_convert(content))
ax1.axis('off')
ax2.imshow(im_convert(style))
ax2.axis('off')

In [0]:
def get_features(image, model):
  layers = {'0':'conv1_1', 
            '5': 'conv2_1',
            '10':'conv3_1', 
            '19':'conv4_1',
            '21': 'conv4_2', #content extraction
            '28': 'conv5_1',
           }
  
  features = {}
  
  for name, layer in model._modules.items():
    image = layer(image)
    if name in layers:
      features[layers[name]] = image
      
  return features

In [0]:
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)

In [0]:
def gram_matrix(tensor):
  batch_size, d, h, w = tensor.size()
  tensor = tensor.view(d, h*w)
  gram = torch.mm(tensor, tensor.t()) # mm multiplies 2 tensors
  return gram

In [0]:
style_gram = {layer: gram_matrix(style_features[layer]) for layer in style_features}

In [0]:
style_weights = {
                  'conv1_1' : 1.0,
                  'conv2_1' : 0.75,
                  'conv3_1' : 0.2,
                  'conv4_1' : 0.2,
                  'conv5_1' : 0.2,
                }

content_weight = 1
style_weight = 1e6

In [0]:
target = content.clone().requires_grad_(True).to(device)


In [0]:
show_every = 300
optimizer = optim.Adam([target], lr = 0.003)
steps = 2100

H, W, C = im_convert(target).shape 
image_array = np.empty(shape=(show_every, H, W, C)) 
counter = 0

In [0]:
for ii in range(1, steps+1):
  target_features = get_features(target, vgg)
  content_loss = torch.mean((target_features['conv4_1'] - content_features['conv4_1'])**2)
  style_loss = 0
  for layer in style_weights:
    target_feature = target_features[layer]
    target_gram = gram_matrix(target_feature)
    style_gram_layer = style_gram[layer]
    layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram_layer)**2)
    _, d, h, w = target_feature.shape
    style_loss += layer_style_loss/ (d * h * w)
    
  loss = content_loss*content_weight + style_loss*style_weight
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  print('Total loss: ', loss.item())
  
  if ii % show_every == 0:
    plt.imshow(im_convert(target))
    plt.axis('off')
    plt.show()
    
  